In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.012457847595214844
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 28.211234827714364
200000 51.4925359692207
300000 56.88252871817227
400000 55.25208240686268
500000 57.175477842133
600000 64.53663954662524
700000 72.72964354744147
800000 68.85815700287202
900000 67.95517245227045
1000000 68.89482725736023
1100000 71.20223565386385
1200000 66.81004646828072
1300000 25.587752844849323
1400000 71.08724674865866
1500000 74.42532553325941
1600000 72.30257628744022
1700000 70.04377057786628
1800000 72.34291503278786
1900000 75.02614198565499
2000000 72.77931286571192
2100000 74.08001542364923
2200000 79.75414817025215
2300000 73.87343056215823
2400000 73.48991341852549
2500000 23.287521552441543
2600000 70.12811440748077
2700000 89.98062003283728
2800000 88.1900440963428
2900000 88.85548648164497
3000000 87.94060199074022
3

30100000 77.18206317522528
30200000 70.92475758198103
30300000 69.61488743497215
30400000 72.75026554108814
30500000 72.48383852290115
30600000 70.0721440361916
30700000 69.13794288510842
30800000 14.234465258783086
30900000 76.00598353724683
31000000 68.44684582087204
31100000 69.37955039302715
31200000 70.92873453657647
31300000 68.48453715035521
31400000 69.33780477277156
31500000 74.63782329658815
31600000 73.82944767920408
31700000 73.78719344980219
31800000 26.790939264867404
31900000 76.4740740252301
32000000 71.93452111387556
32100000 72.78593406704373
32200000 71.28635318903918
32300000 32.25250245320317
32400000 72.50033217198258
32500000 74.00053158659277
32600000 73.01229614371543
32700000 70.68785866615775
32800000 73.60880961482107
32900000 70.62211899632157
33000000 67.31553856755156
33100000 67.29032636292038
33200000 50.868421205635116
33300000 36.93084932374577
33400000 65.17425713725915
33500000 68.32001439615732
33600000 68.19936811894588
33700000 71.09171665947471


60500000 64.8772710163538
60600000 65.11008774803206
60700000 62.41390319924699
60800000 68.63876861351322
60900000 65.55916845703314
61000000 66.86250818968846
61100000 29.026274070832752
61200000 74.57319360044353
61300000 70.17412823016994
61400000 68.81394474268285
61500000 73.26980838170145
61600000 70.79505553756061
61700000 70.13724114414747
61800000 69.81275458975891
61900000 72.021290262305
62000000 36.47512135725051
62100000 60.24727089695207
62200000 68.24105146950359
62300000 68.24520970751213
62400000 64.25719870403434
62500000 63.140731332008656
62600000 60.99995795874253
62700000 63.909328375347954
62800000 18.43185847942739
62900000 64.22821886983087
63000000 63.972362075425806
63100000 61.275723039673046
63200000 60.890024846071945
63300000 69.38719618065399
63400000 65.9683229922334
63500000 48.45547404211409
63600000 51.400446748978936
63700000 69.85989822436397
63800000 72.73482596879107
63900000 68.93957503344723
64000000 70.83276638417044
64100000 76.1571377821811

90800000 69.68085158217792
90900000 69.120623820446
91000000 65.16523934700007
91100000 65.5061799347969
91200000 62.832695825781364
91300000 64.47715485559286
91400000 62.10491376960307
91500000 58.58268732578033
91600000 63.172626094787724
91700000 48.88548002853989
91800000 33.83645162580907
91900000 56.30613655699108
92000000 58.03054794645577
92100000 58.481282115369474
92200000 57.16585377758862
92300000 21.2227381891448
92400000 57.95181791823096
92500000 56.72292127570872
92600000 56.571310253065086
92700000 57.2354460104157
92800000 44.80289337150422
92900000 42.85153705432007
93000000 57.292865724591735
93100000 62.05919521464287
93200000 62.03109282681401
93300000 65.63133240787234
93400000 61.80306442677813
93500000 65.5029370176036
93600000 65.39785248174027
93700000 64.46722694923693
93800000 68.16028433195262
93900000 64.44958211508359
94000000 60.40347905214552
94100000 66.1479248653394
94200000 38.01789041277352
94300000 42.882457521299955
94400000 57.18022787970555
94

120300000 63.70654909292385
120400000 66.42648941847551
120500000 66.82601042639213
120600000 66.76644201229384
120700000 66.84006270103912
120800000 70.72887127354656
120900000 61.11169830819359
121000000 64.46380301569532
121100000 64.61441869834712
121200000 61.064613910200286
121300000 59.96506964127616
121400000 68.23237600039634
121500000 20.625063599100674
121600000 59.357020607123786
121700000 60.47506569421026
121800000 33.45518304781121
121900000 45.24282955078924
122000000 58.80885933676329
122100000 48.57474316960887
122200000 36.91517605256795
122300000 60.25597205890525
122400000 59.478543870123865
122500000 55.632415632623314
122600000 66.21462655306175
122700000 60.09746990645079
122800000 64.87209036663299
122900000 65.15507439263021
123000000 65.20917878777831
123100000 65.13644817908654
123200000 64.85043768101026
123300000 61.7013601935932
123400000 63.46698996898255
123500000 65.77112796056021
123600000 61.40289906178943
123700000 59.103154644706116
123800000 56.21

...

639800000 1.7681161845856017
639900000 1.769597375258843
640000000 1.67280748659901
640100000 1.8011387900613038
640200000 1.8981969219114185
640300000 1.8767937862405815
640400000 1.743980258194828
640500000 1.7161679767608902
640600000 1.6911709224546434
640700000 1.667382148141726
640800000 1.6707343445615928
640900000 1.8070090278650521
641000000 2.0226707801005688
641100000 1.8864330489738443
641200000 1.7042709219758154
641300000 1.647816614080121
641400000 1.6321079597467123
641500000 1.5671086799668492
641600000 1.6407877115782095
641700000 1.7331192598939087
641800000 1.85286986916824
641900000 1.9290608577429853
642000000 1.8219006333024363
642100000 1.6583068191117427
642200000 1.5425738616621527
642300000 1.4959894221686787
642400000 1.5216557813630125
642500000 1.6176142979037305
642600000 1.7022355429487652
642700000 1.8050224798770786
642800000 1.7384964673364605
642900000 1.6655102216428643
643000000 1.598577900757764
643100000 1.491557644848636
643200000 1.403930847231

667400000 2.2438082387430307
667500000 2.206419592766698
667600000 2.2011817840222934
667700000 2.2436032612586234
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.5313670057459068
667900000 0.9653138471684455
668000000 0.9448334244271485
668100000 0.9649682367046429
668200000 0.9807736912037471
668300000 0.9733870843143172
668400000 0.9447427295576467
668500000 0.9343194586819032
668600000 0.9322202038419519
668700000 0.952782943990932
668800000 0.9899307903380336
668900000 0.8892967846850727
669000000 0.8764878819731945
669100000 0.8808072215506274
669200000 0.9162038644464705
669300000 0.9829872239067833
669400000 0.9912107516840181
669500000 0.9017562719637997
669600000 0.9426279995661636
669700000 0.9359850315176189
669800000 1.1271353879658785
669900000 1.1434473393322062
670000000 1.0378728097210685
670100000 1.0231393259329353
670200000 1.0388199389678658
670300000 1.1160596190498548
670400000 1.0896189461498795
670500000 1.0511166704829062
67060

692000000 1.2849031950037069
692100000 1.719785697197991
692200000 1.6601444198120234
692300000 1.2315264557666585
692400000 2.130140348516748
692500000 1.4396491883864067
692600000 1.292871012094691
692700000 2.105367981477536
692800000 1.187497874618123
692900000 1.5751883039617647
693000000 1.7834346539895007
693100000 1.448603911437857
693200000 1.761898631180101
693300000 1.8389582710179266
693400000 1.2801215542009592
693500000 1.8443211136198494
693600000 1.468030591996452
693700000 1.3242231817359627
693800000 1.6708264964774409
693900000 1.590156556536789
694000000 1.5206482562268258
694100000 1.6374656548075206
694200000 1.5466527792175775
694300000 1.6339530112815142
694400000 1.628330831126868
694500000 1.6181495709426026
694600000 1.469582192994551
694700000 1.5856825552252043
694800000 2.0075540839540267
694900000 1.5157523227112888
695000000 1.4572317134271504
695100000 1.7506995446656048
695200000 1.520179513526509
695300000 1.5059353332599141
695400000 1.42162616616724

713200000 0.10370105996337135
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.08950442004647917
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.10405860320425298
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 135
cases of this type: 1215
best so far: 0
type: [1, 27, 1, 1, 5] 135
cases of this type: 16875
713500000 0.10635688288486997
best

best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.00032909199953344136
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases 